In [48]:
# magic! (don't worry about this)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
# let us import some useful things
from lib import *
from classifiers import *
import math

In [72]:
# load the data same as before
microarray_file_name = 'data/leukemia_ALL_AML_matrix.txt'
labels_file_name = 'data/leukemia_ALL_AML_labels.txt'
data_store = DataSet(microarray_file_name, labels_file_name)

In [74]:
# instantiate the kNN classifier
k = 3
knn = KNearestNeighbors()
knn.train(data_store.get_train_set()) # provide training samples
classified_samples = knn.classify(data_store.get_test_set(), k, euclidean_distance)

# let's evaluate how well the classifier worked
evaluate_results(classified_samples)

Accuracy: 84.00%


In [59]:
# TODO: modify the code above to evaluate the classifier with different values of k (1 to 10)

In [ ]:
# now let's start writing the kNN classifier code in the classifiers_skeleton.py file

# TODO: change the import statement above to import classifiers_skeleton instead
 
# TODO: take a look at the evaluate_results function (this is implemented for you)
# can you figure out what is the type of the input argument classified_samples?
# what happens in the first line of this function? how about the third line?

# what happens when you run the skeleton code? what is the accuracy of this partially-implemented classifier? why? 

# TODO: fill in each TODO section in the file!

# congratulations, you have now implemented a kNN classifier!